In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [2]:
import os
filename = []
list = os.listdir('/Users/HP/Documents/Cap/capstone/notebooks')
for f in list:
    if "csv" in f:
        filename.append(f)

In [3]:
filename

['1814.csv', '1830.csv', '1834.csv', '1848.csv', '1901_fb.csv', '1918_fb.csv']

In [4]:

df = pd.DataFrame()
for f in filename:
    df_c = pd.read_csv(f)
    df=df.append(df_c)

    

In [5]:
features = df[['hr', 'pat']]
target = df[['sbp', 'dbp']]

In [6]:
from sklearn import preprocessing
fm_norm = preprocessing.scale(features)

In [7]:
fm_norm.shape

(83, 2)

In [8]:
print(target)
print(target.values)

           sbp        dbp
0   106.537500  80.002308
1   111.934615  82.893077
2   114.816154  84.943077
3   113.557857  85.763846
4   106.041333  81.635333
..         ...        ...
17   86.661538  54.647857
18   86.303077  54.577500
19   85.340909  53.865000
20   82.847500  52.957000
21   81.151667  53.101429

[83 rows x 2 columns]
[[106.5375      80.00230769]
 [111.93461538  82.89307692]
 [114.81615385  84.94307692]
 [113.55785714  85.76384615]
 [106.04133333  81.63533333]
 [100.555625    77.40266667]
 [101.326       75.17571429]
 [102.80153846  74.42583333]
 [108.237       75.699     ]
 [115.03166667  79.14333333]
 [114.50166667  83.78833333]
 [112.99666667  83.845     ]
 [109.9         82.60333333]
 [106.49923077  80.58307692]
 [101.49933333  76.12642857]
 [ 99.84        76.454     ]
 [ 99.80071429  76.685     ]
 [100.6475      76.69615385]
 [103.57444444  78.02      ]
 [104.40333333  76.018     ]
 [120.54083333  82.35083333]
 [125.15666667  86.99583333]
 [127.32        89.42333333

In [40]:
def cindex(true_labels, predictions):
    n,h = 0,0
    
    for i in range(1,len(true_labels)):
        t = true_labels[i]
        p = predictions[i]
        for j in range(i+1,len(true_labels)):
            nt = true_labels[j]
            np = predictions[j]
            if t != nt:
                n += 1
                if (p < np and t < nt) or (p > np and t > nt):
                    h += 1
                else :
                    h += 0.5
                    ci = h/n
    return ci

In [41]:
cindex([1,1,1,4,5,8], [1,2,3,5,5,8])


0.9285714285714286

In [42]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
loo = LeaveOneOut()
X = fm_norm
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []

for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")

Average MAE SBP: 8.628385398873796 MAE DBP: 9.111342488722784
C index SBP: 0.8029208069858477  C index DBP: 0.763855421686747


In [48]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators = 80)
loo = LeaveOneOut()
X = fm_norm
y = target.values
sbp_er = []
dbp_er = []
true_label = []
pred_label = []

for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    

    forest.fit(X_train, y_train)
    y_pred = forest.predict(X_test)
    
    mae_sbp = mean_absolute_error(y_pred[:,0], y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred[:,1], y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    true_label.append(y_test)
    pred_label.append(y_pred)
true = [item for sublist in true_label for item in sublist]
pred = [item for sublist in pred_label for item in sublist]

true = np.array(true)
pred = np.array(pred)

cidx_sbp = cindex(true[:,0], pred[:,0])
cidx_dbp = cindex(true[:,1], pred[:,1])
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")


Average MAE SBP: 8.998776773615443 MAE DBP: 9.72908250345078
C index SBP: 0.7925323697681421  C index DBP: 0.7617469879518072


In [52]:
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf', C=10, epsilon = 0.01)
X = fm_norm
y = target.values
sbp_er = []
dbp_er = []
true_label_sbp = []
pred_label_sbp = []

true_label_dbp = []
pred_label_dbp= []

for i, (train_idx, val_idx) in enumerate(loo.split(X)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]
    

    svr.fit(X_train, y_train[:,0])
    y_pred_sbp = svr.predict(X_test)
    true_label_sbp.append(y_test[:,0])
    pred_label_sbp.append(y_pred_sbp)
    
    svr.fit(X_train, y_train[:,1])
    y_pred_dbp = svr.predict(X_test)
    true_label_dbp.append(y_test[:,1])
    pred_label_dbp.append(y_pred_dbp)
    
    mae_sbp = mean_absolute_error(y_pred_sbp, y_test[:,0])
    mae_dbp = mean_absolute_error(y_pred_dbp, y_test[:,1])
    
    sbp_er.append(mae_sbp)
    dbp_er.append(mae_dbp)
    
    
    
true_sbp = [item for sublist in true_label_sbp for item in sublist]
pred_sbp = [item for sublist in pred_label_sbp for item in sublist]

true_dbp = [item for sublist in true_label_dbp for item in sublist]
pred_dbp = [item for sublist in pred_label_dbp for item in sublist]

true_sbp = np.array(true_sbp)
pred_sbp = np.array(pred_sbp)

true_dbp = np.array(true_dbp)
pred_dbp = np.array(pred_dbp)

cidx_sbp = cindex(true_sbp, pred_sbp)
cidx_dbp = cindex(true_dbp, pred_dbp)
    
print(f"Average MAE SBP: {np.mean(sbp_er)} MAE DBP: {np.mean(dbp_er)}")
print(f"C index SBP: {cidx_sbp}  C index DBP: {cidx_dbp}")

Average MAE SBP: 7.806436839221576 MAE DBP: 8.18428939099958
C index SBP: 0.8152664859981933  C index DBP: 0.784487951807229
